# Multi-objective Robust Optimization (MORO)


This exercise demostrates the application of MORO on the lake model. In contrast to the exercises in previous weeks, we will be using a slightly more sophisticated version of the problem. For details see the MORDM assignment for this week.

## Setup MORO

Many objective robust optimization aims at finding decisions that are robust with respect to the various deeply uncertain factors. For this, MORO evalues each candidate decision over a set of scenarios. For each outcome of interest, the robusntess over this set is calculated. A MOEA is used to maximize the robustness. 

For this assignment, we will be using a domain criterion as our robustness metric. The table below lists the rules that you should use for each outcome of interest.

|Outcome of interest| threhsold  |
|-------------------|------------|
| Maximum pollution | $\leq$ 0.75|
| Inertia           | $\geq$ 0.6 |
| Reliability       | $\geq$ 0.99|   
| Utility           | $\geq$ 0.75|

**1) Implement a function for each outcome that takes a numpy array with results for the outcome of interest, and returns the robustness score**

In [6]:
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model)
import functools

def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

def maxp(data):
    return np.sum(data<=0.75)/data.shape[0]
    
SMALLER = 'SMALLER'
LARGER = 'LARGER'

maxp = functools.partial(robustness, SMALLER, 0.75)
inertia = functools.partial(robustness, LARGER, 0.6)
reliability = functools.partial(robustness, LARGER, 0.99)
utility = functools.partial(robustness, LARGER, 0.75)

MAXIMIZE = ScalarOutcome.MAXIMIZE
MINIMIZE = ScalarOutcome.MINIMIZE
# choose one, not sure which.  
robustnes_functions = [ScalarOutcome('90th percentile max_p', kind=MINIMIZE,
                             variable_name='max_P', function=maxp),
                       ScalarOutcome('10th percentile reliability', kind=MAXIMIZE,
                             variable_name='reliability', function=reliability),
                       ScalarOutcome('10th percentile inertia', kind=MAXIMIZE,
                             variable_name='inertia', function=inertia),
                       ScalarOutcome('10th percentile utility', kind=MAXIMIZE,
                             variable_name='utility', function=utility)]

robustnes_maxp_function = [ScalarOutcome('max_p', kind=MINIMIZE,
                             variable_name='max_P', function=maxp)]
robustnes_reliability_function = [ScalarOutcome('reliability', kind=MAXIMIZE,
                             variable_name='reliability', function=reliability)]
robustnes_inertia_function = [ScalarOutcome('inertia', kind=MAXIMIZE,
                             variable_name='inertia', function=inertia)]
robustnes_utility_function = [ScalarOutcome('utility', kind=MAXIMIZE,
                             variable_name='utility', function=utility)]




**2) Generate 4 random release policies, and evaluate them over 500 scenarios. Sample the scenarios using Monte Carlo sampling. Next evaulate your robustness function for 1, 2, 3, ... 500 scenarios for each outcome and visualize this. What can you tell about the convergernce of the robusntess metric as a function of the number of scenarios?**

In [8]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments)
from ema_workbench.em_framework.evaluators import MC
from dps_lake_model import lake_model as lakeModel
lake_model = Model('lakeproblem', function=lakeModel)
# instantiate model
lake_model.uncertainties = [RealParameter('b', 0.1, 0.45),
                                RealParameter('q', 2.0, 4.5),
                                RealParameter('mean', 0.01, 0.05),
                                RealParameter('stdev', 0.001, 0.005),
                                RealParameter('delta', 0.93, 0.99)]

# set levers
lake_model.levers = [RealParameter("c1", -2, 2),
                         RealParameter("c2", -2, 2),
                         RealParameter("r1", 0, 2),
                         RealParameter("r2", 0, 2),
                         RealParameter("w1", 0, 1)
                         ]

# specify outcomes
lake_model.outcomes = [ScalarOutcome('max_P'),
                           ScalarOutcome('utility'),
                           ScalarOutcome('inertia'),
                           ScalarOutcome('reliability')]

# override some of the defaults of the model
lake_model.constants = [Constant('alpha', 0.41),
                            Constant('nsamples', 100),
                            Constant('myears', 100)]
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=500, policies=4, uncertainty_sampling=MC)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 500 scenarios * 4 policies * 1 model(s) = 2000 experiments
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [12]:
experiments, outcomes = results
maxp_exp = experiments[experiments['policy']==0]
experiments, outcomes = results
utility_exp = experiments[experiments['policy']==1]
experiments, outcomes = results
reliability_exp = experiments[experiments['policy']==2]
experiments, outcomes = results
inertia_exp = experiments[experiments['policy']==3]

b     delta      mean         q     stdev        c1        c2  \
0    0.235782  0.946745  0.037606  3.848933  0.001201  1.273938 -0.337456   
1    0.111899  0.984623  0.034387  3.762675  0.004325  1.273938 -0.337456   
2    0.298016  0.959069  0.038599  3.970124  0.002031  1.273938 -0.337456   
3    0.233245  0.933301  0.037443  2.548092  0.003673  1.273938 -0.337456   
4    0.208404  0.984000  0.037064  4.154736  0.001448  1.273938 -0.337456   
..        ...       ...       ...       ...       ...       ...       ...   
495  0.447434  0.973819  0.037656  3.028653  0.002486  1.273938 -0.337456   
496  0.236288  0.977609  0.043207  3.704230  0.001577  1.273938 -0.337456   
497  0.166828  0.988848  0.021256  3.436432  0.004271  1.273938 -0.337456   
498  0.336323  0.952599  0.016970  2.839700  0.002878  1.273938 -0.337456   
499  0.391520  0.946084  0.048156  3.350823  0.002582  1.273938 -0.337456   

           r1        r2       w1 scenario policy        model  
0    0.875933  1.921154

In [4]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments)
from ema_workbench.em_framework.evaluators import MC
from ema_workbench.em_framework.samplers import sample_uncertainties
from dps_lake_model import lake_model as lakeModel

n_scenarios = 500
scenarios = sample_uncertainties(lake_model, n_scenarios, sampler=MC)
nfe=1000

with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.robust_optimize(scenarios=500, policies=4, uncertainty_sampling=MC)


## Searching for candidate solutions
Set up the robust optimization problem using the robustness functions you have specified. Assume that you will need 50 scenarios for estimating the robustness. Use $\epsilon$-progress and hypervolume to track convergence. Solve the optimization problem. As $\epsilon$ values, you can assume 0.05 for each of the four robustness metrics.

*note: this optimization problem is computationally very expensive. Develop and test your code using a sequential evaluator, a low number of function evaluations (e.g., 200), and a low number of scenarios (e.g., 5). Once everything seems to be working replace the sequential evaluator with an multiprocessing or ipyparallel evaluator, and increase the number of nfe and scenarios*.


**Plot your $\epsilon$-progress to evaluate convergergence, and visualize the trade-offs using parallel coordinate plots**

**What does this plot tell us about the tradeoffs and conflicting objectives?**

## Re-evaluate candidate solutions under uncertainty

We have used only 50 scenarios for the optimization. Take the results and re-evaluate them over a larger set (assume 1000 scenarios). How different are your results? What does this imply for the assumption of 50 scenarios during robust optimization.

*hint: use the to_dict method on a dataframe, next generate Policy objects in a list expression by iterating over the dicts returned by the to_dict method*

## Comparison
If you have time, import your solutions found for MORDM and re-evaluate them over the same set of scnearios as used for re-evaluating the MORO results. Compare the robustness of MORDM and MORO, what do you observe?